In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame
from torchvision.models import VGG16_Weights, vgg16
from torchvision.utils import save_image
import torch
import torchvision
from openTSNE import TSNE
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import os

from mapping import layer_idx_to_model_layer_idx
from process_annotations import mapping_label_id_to_category, mapping_category_to_label
import process_annotations
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import os

In [9]:
def create_directory_structure(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    create_directory(images_dir)

    #create a directory for every layer , 36 & 37, are input and output 
    for layer_idx in range(38):
        
        layer_dir = os.path.join(data_dir, str(layer_idx))
        create_directory(layer_dir)
        
        if layer_idx == 37:
            top10_dir  = os.path.join(layer_dir, 'top10')
            create_directory(top10_dir)
        if layer_idx == 36:
            images_dir  = os.path.join(layer_dir, 'images')
            create_directory(images_dir)
        
        
        
        activations_dir  = os.path.join(layer_dir, 'activations')
        create_directory(activations_dir)
        
        activations_images =  os.path.join(activations_dir, 'images')
        create_directory(activations_images)

        activations_raw_data = os.path.join(activations_dir, 'raw_data')
        create_directory(activations_raw_data)
    

#via chatgpt
def create_directory(path):
    # Specify the directory name
    directory_name = path

    # Create the directory
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists")
    except Exception as e:
        print(f"Error: {e}")

    pass
    
create_directory_structure('assets/data/imagenette/700')

Directory 'assets/data/imagenette/700/images' created successfully
Directory 'assets/data/imagenette/700/0' created successfully
Directory 'assets/data/imagenette/700/0/activations' created successfully
Directory 'assets/data/imagenette/700/0/activations/images' created successfully
Directory 'assets/data/imagenette/700/0/activations/raw_data' created successfully
Directory 'assets/data/imagenette/700/1' created successfully
Directory 'assets/data/imagenette/700/1/activations' created successfully
Directory 'assets/data/imagenette/700/1/activations/images' created successfully
Directory 'assets/data/imagenette/700/1/activations/raw_data' created successfully
Directory 'assets/data/imagenette/700/2' created successfully
Directory 'assets/data/imagenette/700/2/activations' created successfully
Directory 'assets/data/imagenette/700/2/activations/images' created successfully
Directory 'assets/data/imagenette/700/2/activations/raw_data' created successfully
Directory 'assets/data/imagenette

### Functions

In [5]:
def calculate_score(data_2D):
    #calculate the entropy
    entropy = calculate_binned_entropy(data_2D)
    
    
    #calculate the magnitude, overall mean:
    magnitude = np.mean(data_2D)

    score = entropy * abs(magnitude)
    
    return entropy, magnitude, score

def calculate_scoring(data):
    scoring = []
    for channel in range(len(data)):
        data_2D = data[channel]
        scoring.append(calculate_score(data_2D))
    return scoring

#via chatgpt
def min_max_normalization(array):
    """Normalize a 2D array to the range [0, 1] using Min-Max scaling."""
    return (array - np.min(array)) / (np.max(array) - np.min(array))

#via chatgpt
def calculate_binned_entropy(data, bins=10):
    """
    Calculate the entropy of a 2D array using Min-Max normalization and binning.
    
    Parameters:
        data (numpy array): 2D array of activations.
        bins (int): Number of bins for discretization.
    
    Returns:
        float: Entropy value.
    """
    # Step 1: Normalize the data to [0, 1]
    normalized_data = min_max_normalization(data)
    
    # Step 2: Flatten the array for entropy calculation
    flattened = normalized_data.flatten()
    
    # Step 3: Create histogram bins and count occurrences
    hist, bin_edges = np.histogram(flattened, bins=bins, density=True)
    
    # Step 4: Normalize histogram to get probabilities
    probabilities = hist / hist.sum()
    
    # Step 5: Compute entropy using the Shannon formula
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-12))  # Avoid log(0) by adding a small value
    
    return entropy


In [19]:

def save_activation_to_image(data, data_dir, layer_idx, uid, scoring):
    
    for channel in range(len(data)):
        entropy = scoring[channel][0]
        magnitude = scoring[channel][1]
        score = scoring[channel][2]
        
        
        
        activation = data[channel]
        
        activation_np = np.array(activation, dtype=np.uint8)
        
        normalized_activation_np = 1000 * (activation_np - np.min(activation_np)) / (np.max(activation_np) - np.min(activation_np))
        normalized_activation_np = normalized_activation_np.astype(np.uint8)
        
        cmap = plt.cm.plasma  # You can choose other colormaps like 'plasma', 'inferno', etc.
        rgb_image = cmap(normalized_activation_np)
        
        normalized_rgb_image = 255 * (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))
        normalized_rgb_image = normalized_rgb_image.astype(np.uint8)
        normalized_rgb_image = normalized_rgb_image[:,:, :3]
        
        # The result is an RGBA image (last channel is alpha), so remove the alpha channel
        #rgb_image = (normalized_array[:, :, :3]).astype(np.uint8)
        
        # Save the image using Pillow
        image = Image.fromarray(normalized_rgb_image)
        path = os.path.join(data_dir, f'{layer_idx}/activations/images/{layer_idx}_{uid}_{channel}_{entropy}_{magnitude}_{score}.png')
        image.save(path)
        print(f'saved: {path}')
    
    pass

def save_activation_to_array(data, data_dir, layer_idx, uid, scoring):
    
    for channel in range(len(data)):
        
        entropy = scoring[channel][0]
        magnitude = scoring[channel][1]
        score = scoring[channel][2]
        
        activation = data[channel]
        path = os.path.join(data_dir, f'{layer_idx}/activations/raw_data/{layer_idx}_{uid}_{channel}_{entropy}_{magnitude}_{score}.npy')
        np.save(path, activation)
        print(f'saved: {path}')
    pass


### Generate data

In [8]:

data_dir = 'assets/data/imagenette/700'

# Step 1: Initialize the data

dataiter = process_annotations.get_data_iter_imagenette(1)
image, label, label_id, category, uid, img_path = next(dataiter)
print(img_path)
uid = uid[0].item()
#sample_dir = os.path.join(data_dir, str(uid))

image_input = image[0].unsqueeze(0)
#print(label_id)
#print(image_input)

#Step 2: save input layer data

#save orginal image
shutil.copy2(img_path[0], os.path.join(data_dir,f'images/{uid}.png'))

#save full image, normalized passed into the network
image_path = os.path.join(data_dir, f'36/images/{uid}.png')
save_image(image, image_path)


#calculate score, for each channel
scoring = calculate_scoring(image)

#save activations rgb
save_activation_to_image(image, data_dir, 36, uid, scoring)
#save activation raw data
save_activation_to_array(image, data_dir, 36, uid, scoring)



# Step 3: load model

weights = VGG16_Weights.DEFAULT
model = vgg16(weights=weights)
model.eval()

#print(dict(model.named_modules()))

#Initialize the inference transforms
preprocess = weights.transforms()



# Step 4: Register hooks

activations = {}

def hook_fn(module, input, output, layer_idx):
    print(f"Hook triggered for: {layer_idx}, {module.__class__.__name__}")
    activations[layer_idx] = output

#register conv part
for layer_idx in range(31):
    layer = model.features[layer_idx_to_model_layer_idx[layer_idx]]    
    layer.register_forward_hook(lambda module, input, output, layer_idx_hook=layer_idx: hook_fn(module, input, output, layer_idx_hook))
    
#register neuron part
for layer_idx in range(31,36):
    layer = model.classifier[layer_idx_to_model_layer_idx[layer_idx]]
    layer.register_forward_hook(
        lambda module, input, output, layer_idx_hook=layer_idx: hook_fn(module, input, output, layer_idx_hook))


# Step 5 run inference for current image

with torch.no_grad():
    out = model(image_input)


# Step 6 classify image, save top 5, save target 37

output_top10_dir = os.path.join(data_dir, str(37) + '/top10')
probabilities = torch.nn.functional.softmax(out[0], dim=0)
top10_prob, top10_catid = torch.topk(probabilities, 10)
categories = [mapping_label_id_to_category[cat_id.item()] for cat_id in top10_catid]
labels = [mapping_category_to_label[category] for category in categories]
top10 = {
    'uid': [uid] * 10,
    'gt_label_id': [label_id[0].item()] * 10,
    'label_id': top10_catid ,
    'category': categories,
    'label': labels,
    'prob': top10_prob
}

print(top10)

top10_path = os.path.join(output_top10_dir, f'top10_{uid}.csv')
top10_df = pd.DataFrame(data=top10)
top10_df.to_csv(top10_path)


#Save the output activation raw values

raw_activation = {
    'uid': [uid] * 1000,
    'idx': list(range(1000)),
    'value': probabilities
}

output_raw_activation_path = os.path.join(data_dir, str(37) + f'/activations/raw_data/37_{uid}.csv')
raw_activation_df = pd.DataFrame(data=raw_activation)
raw_activation_df.to_csv(output_raw_activation_path)

# Step 7 save  activation conv data


# Step 8 save activation classification data


# Step 8 save output activation




#todo do classification
#print(activations)
        

Loading dataset
Done loading datset
('datasets/imagenette2-320/train/n03028079/n03028079_4086.JPEG',)


TypeError: 'int' object is not iterable

In [8]:
top10_df


,uid,label_id,category,label,prob
0,6594,566,,,9.998535e-01
1,6594,513,,,9.126405e-05
2,6594,875,,,4.590953e-05
3,6594,432,,,6.682266e-06
4,6594,683,,,2.477643e-06
5,6594,776,,,1.370156e-07
6,6594,558,,,1.625363e-08
7,6594,652,,,5.779195e-09
8,6594,439,,,2.324000e-09
9,6594,541,,,1.656787e-09


In [ ]:
top10_df

In [ ]:
#show figure



In [5]:
 model.classifier
 

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)